# R语言进阶



## 简介

在本讲中，我们将会讨论如何在R中进行高效运算。

* R 是一个向量导向的语言。大多数情况下，向量化将会加快运算的速度。
* 如果没有办法通过优化代码以提高速度，我们可以使用更多的 CPU 进行并行执行，以节省时间。
* 如何在远程访问服务器。远程集群连接与在本地操作计算机并不相同。

## 向量化

虽然有多种在数学上等效的计算方法，但是在运算速度上它们存在着明显的差异。

运算速度真的那么重要吗?对于一个运算时间不到一分钟的项目来说，效率的差别并不显著。但有时候一个经济学的问题会非常冗杂。比如常见的结构化估计，单个的估计就可能需要一周的时间来运算。在计量经济学中，其他的计算密集型算法，比如自举法 (bootstrap) ，模拟最大似然估计 (simulated maximun likehood，简记 sml) 和模拟矩估计 (simulated method of moments) 等， 即使它们单次执行不需要太多的时间，重复上千次这样的流程也会占用非常多的时间。

不仅如此，机器学习方法通常会涉及到调谐参数，而网格上每一点的调谐参数都需要重复相同的步骤。例如{cite}`lin2020`里的算法在一个24核的远程服务器中运算了8小时才找出了调谐参数的最佳组合。在这些问题中，优化代码是一个必要步骤。

当然，优化代码需要投入开发者的时间。这是人力时间和机器时间的取舍与平衡问题。


```{prf:example}

在同方差的OLS回归中

$$
\sqrt{n}\left(\widehat{\beta}-\beta_{0}\right)\stackrel{d}{\to}N\left(0,\sigma^{2}\left(E\left[x_{i}x_{i}'\right]\right)^{-1}\right)
$$

其中渐进方差可以通过 $(X'X)^{-1} \sum_{i=1}^n \widehat{e}^{2}$ 相合估计。然而，如果是异方差，则

$$
\sqrt{n}\left(\widehat{\beta}-\beta_{0}\right)\stackrel{d}{\to}N\left(0,E\left[x_{i}x_{i}'\right]^{-1}\mathrm{var}\left(x_{i}e_{i}\right)E\left[x_{i}x_{i}'\right]^{-1}\right)
$$

其中 $\mathrm{var}\left(x_{i}e_{i}\right)$ 可以用下式估计 

$$
\underset{\mathrm{opt1}}{\frac{1}{n}\sum_{i=1}^{n}x_{i}x_{i}'\widehat{e}_{i}^{2}}=\underset{\mathrm{opt2,3}}{\frac{1}{n}X'DX}=\underset{\mathrm{opt 4}}{\frac{1}{n}\left(X'D^{1/2}\right)\left(D^{1/2}X\right)}
$$

 $D$ 是 $\left(\widehat{\epsilon}_{1}^{2},\widehat{\epsilon}_{2,}^{2},\ldots,\widehat{\epsilon}_{n}^{2}\right)$ 的对角矩阵。
要计算三明治形式 (the sandwich form) 的“肉”，至少有四种等效的方法：

1. 直接基于 $i=1,\ldots,n$ 一个个加总 $\hat{e}_i^2 x_i x_i'$ 。
2. 使用稠密的中心矩阵 $X' \mathrm{diag}(\hat{e}^2) X$ .
3. 使用稀疏的中心矩阵 $X' \mathrm{diag}(\hat{e}^2) X$ . 
4. 对 `X*e_hat` 进行叉乘 (cross product) 。它可以用R语言进行逐个运算。 


首先，我们生成二元响应 (binary regressor) 和回归变量的数据。由于因变量是离散的，线性概率模型中的误差项是异方差。 所以，我们必须用异方差稳健方差 (heteroskedastic-robust variance) 才能相合估计OLS的渐近方差。使用下方的代码就能估计模型的系数，并计算对应的残差。
```

In [ ]:
# an example of robust variance matrix.
# compare the implementation via matrix, Matrix (package) and vecteroization.

# n = 5000; Rep = 10; # Matrix is quick, matrix is slow, adding is OK

source("data_example/lec2.R")

n <- 50
Rep <- 1000 

data.Xe <- lpm(n) # see the function in "data_example/lec2.R"
X <- data.Xe$X
e_hat <- data.Xe$e_hat

XXe2 <- matrix(0, nrow = 2, ncol = 2)


对于相同的数据，我们对于四种方式分别运算，并比较每种方法的时间。

In [ ]:
for (opt in 1:4) {
  pts0 <- Sys.time()

  for (iter in 1:Rep) {
    set.seed(iter) # to make sure that the data used
    # different estimation methods are the same


    if (opt == 1) {
      for (i in 1:n) {
        XXe2 <- XXe2 + e_hat[i]^2 * X[i, ] %*% t(X[i, ])
      }
    } else if (opt == 2) { # the vectorized version with dense matrix
      e_hat2_M <- matrix(0, nrow = n, ncol = n)
      diag(e_hat2_M) <- e_hat^2
      XXe2 <- t(X) %*% e_hat2_M %*% X
    } else if (opt == 3) { # the vectorized version with sparse matrix
      e_hat2_M <- Matrix::Matrix(0, ncol = n, nrow = n)
      diag(e_hat2_M) <- e_hat^2
      XXe2 <- t(X) %*% e_hat2_M %*% X
    } else if (opt == 4) { # the best vectorization method. No waste
      Xe <- X * e_hat
      XXe2 <- t(Xe) %*% Xe
    }


    XX_inv <- solve(t(X) %*% X)
    sig_B <- XX_inv %*% XXe2 %*% XX_inv
  }
  cat("n =", n, ", Rep =", Rep, ", opt =", opt, ", time =", Sys.time() - pts0, "\n")
}


尽管从数学上来说，四种方法都能计算出结果，但是我们可以明显地看出运算时间的差异。
当 $n$ 很小时， `matrix` 较快而 `Matrix` 较慢；向量化的方式是最快的。
当 $n$ 很大时， `matrix` 较慢而 `Matrix` 较快；向量化的方法依然是最快的。

因为这次的模型很简单，单次运行仅仅占用非常短暂的时间，所以在这次测试中，我们多次重复相同的流程以更直观地体现效率的差距。而复杂的模型，例如 `data_example/IPUMS.R` 有23.4万的观测值，用不同方法计算就可能有惊人的时间差。这展现了向量化的重要作用。在复杂的运算中，向量化可以有效节省计算时间。例如，在异方差自回归的相合方差估计量(HAC)中，就涉及到了多层的矩阵运算。


## 高效循环

R来源于S语言，一门“古老”的语言。因此，R语言中有许多拓展包以适应现在的大数据时代。{cite}`wickham2016r` 中就有我们所需的例子。 
在此我们介绍 [`plyr`](http://plyr.had.co.nz/)。

在传统的 `for` 循环中，我们不得不处理各种琐碎的工作。但 [Hadley Wickham](http://had.co.nz/) 的 `plyr` 简化了工作并支持**循环并行化** (parallelization) 。

```{prf:example}

在这里，我们计算一个自由度为2的泊松分布(Poisson distribution)的实证收敛概率。首先，我们写一个自定义函数`CI`来运算置信区间。(这在上一讲中已经使用过)

这是一个经典的 `for` 循环
```

In [ ]:
Rep <- 100000
sample_size <- 1000
mu <- 2

source("data_example/lec2.R")
# append a new outcome after each loop
pts0 <- Sys.time() # check time
for (i in 1:Rep) {
  x <- rpois(sample_size, mu)
  bounds <- CI(x)
  out_i <- ((bounds$lower <= mu) & (mu <= bounds$upper))
  if (i == 1) {
    out <- out_i
  } else {
    out <- c(out, out_i)
  }
}

pts1 <- Sys.time() - pts0 # check time elapse
cat("loop without pre-definition takes", pts1, "seconds\n")


# pre-define a container
out <- rep(0, Rep)
pts0 <- Sys.time() # check time
for (i in 1:Rep) {
  x <- rpois(sample_size, mu)
  bounds <- CI(x)
  out[i] <- ((bounds$lower <= mu) & (mu <= bounds$upper))
}

pts1 <- Sys.time() - pts0 # check time elapse
cat("loop with pre-definition takes", pts1, "seconds\n")


注意这一行 `out = rep(0, Rep)`。 它提前定义( pre-defines )了一个 `out` 向量。而 `out` 向量则通过
`out[i] = ( ( bounds$lower <= mu  ) & (mu <= bounds$upper) )`赋值。为了向量`out`的计算，电脑打开一个连续的内存补丁。当输入新结果的时候，就会替换掉旧元素 (element) 。如果我们不提前定义`out`，而是在每个循环中添加 (append) 一个新元素，这样在每次循环中`out`向量的长度会随之改变，而且每一次都需要分配 (assign) 一个新的内存补丁来储存它。在第二个方法中，仅仅在内存中定位所需的向量，就要多占用大量的时间。

`out` 是结果的储存器。在一个 `for` 循环中，我们提前定义一个结果的储存向量；这样在每一次的循环中，我们就可以直接调用索引来替换对应的元素。

相反，一个 `plyr` 循环不需要处理这些琐碎的细节，还可以快捷进行循环并行运算。下方的例子中，我们将`for`循环的代码封装为函数 `capture`，然后通过 `__ply` 指令重复运行。`__ply` 是一系列的函数。这里的 `ldply` 的含义是循环的输入是一个列表(`l`)，而输出则是一个数据框(`d`)。


In [ ]:
library(plyr)

capture <- function(i) {
  x <- rpois(sample_size, mu)
  bounds <- CI(x)
  return((bounds$lower <= mu) & (mu <= bounds$upper))
}

pts0 <- Sys.time() # check time
out <- ldply(.data = 1:Rep, .fun = capture)

pts1 <- Sys.time() - pts0 # check time elapse
cat("plyr loop takes", pts1, "seconds\n")


这个例子比较简单，没有办法凸显 `plyr` 的优势。在处理大量数据的复杂问题时，不同代码带来的效率区别就十分明显了。
单论速度， `plyr` 并没有比 `for` 循环快上多少。它们的表现不分伯仲。 我们接下来要介绍的是并行运算。如果使用`plyr`，这将会非常容易——我们仅需改变函数中的一个 **argument** 就可以了。

## 并行运算

当样本规模超出单个计算器的储存能力时，并行运算就体现了它的必要性。{cite}`li2018embracing`就是一个不错的例子。
在这里，我们研究在多核计算器中进行并行运算能够获得多少的速度提升。首先，我们会介绍如何在单个计算器中协调多个核 (cores)。
 `foreach` 和 `doParallel` 都是并行运算常用的拓展包。
下面的代码就是并行运算的基础结构。 `registerDoParallel(number)` 为后续并行运算提前准备了特定数量的CPU核。


In [ ]:
library(plyr); library(foreach); library(doParallel)

registerDoParallel(a_number) # opens specified number of CPUs

out <- foreach(icount(Rep), .combine = option) %dopar% {
  my_expressions
}


理论上如果我们有两个自行运转的CPU，我们可以比单个CPU缩减一半的时间。那么事实是否与理论一致呢？

```{prf:example}
比较单核顺序循环和并行循环的运算速度
```

In [ ]:
library(foreach)
library(doParallel)

registerDoParallel(2) # open 2 CPUs

pts0 <- Sys.time() # check time

out <- foreach(icount(Rep), .combine = c) %dopar% {
  capture()
}

pts1 <- Sys.time() - pts0 # check time elapse
cat("parallel loop takes", pts1, "seconds\n")


出乎意料的是，上方的代码运行得甚至更慢了。实际上，这是因为每一次循环都可以在非常短的时间内处理完毕。下方的代码则与之相反。 每一次循环花费的时间都是非平凡的 (non-trivial) 。而两个代码的区别仅仅是第一个使用的是 `%dopar%` 而第二个使用了 `%do%`。

In [ ]:
Rep <- 200
sample_size <- 2000000

registerDoParallel(8) # change the number of open CPUs according to
# the specification of your computer

pts0 <- Sys.time() # check time
out <- foreach(icount(Rep), .combine = c) %dopar% {
  capture()
}

cat("8-core parallel loop takes", Sys.time() - pts0 , "seconds\n")

pts0 <- Sys.time()
out <- foreach(icount(Rep), .combine = c) %do% {
  capture()
}

cat("single-core loop takes", Sys.time() - pts0 , "seconds\n")



## 拓展阅读

Wickham and Grolemund: 第3、10、11、21以及26-30章


## 参考文献
